In [1]:
# -*- coding: utf-8 -*-
"""
Spyder Editor
This is a temporary script file.
"""

import os
import sys
import subprocess

import atexit
import numpy as np
import tempfile
from multiprocessing import Process, Queue

In [2]:
#gisdb = os.path.join(os.path.expanduser("~"), "grassData2/FUTURES_triangle")


gisdb = '/cgeo_home/rlopez3'

#location = "futures_ncspm"

location  = "BrazilREDD"

mapset = "PERMANENT"

# path to the GRASS GIS launch script
# we assume that the GRASS GIS start script is available and on PATH
# query GRASS itself for its GISBASE
# (with fixes for specific platforms)
# needs to be edited by the user

In [3]:

#devpressure_0_5_92	road_dens_perc	forest_1992_smooth_perc	dist_to_water_km	dist_to_protected_km

# define GRASS Database
# add your path to grassdata (GRASS GIS database) directory
# specify (existing) Directory, Location and Mapset

'''
Example locating GrassGisDatabase directory, Grass Location and grass mapset:
-------
*this is the database grass gis directory where inside is the grass location futures_triange***
(base) rodrigo@rodrigo-centromet:~/grassData2/FUTURES_triangle$ ls
futures_ncspm  futures_triangle_files
The mapsets are found in Futures
(base) rodrigo@rodrigo-centromet:~/grassData2/FUTURES_triangle$ ls futures_ncspm/
CREDITS.txt  FUTURES_triangle  PERMANENT                  population_trend.csv  VERSION.txt
demand.csv   HISTORY.txt       population_projection.csv  practice1
'''


#gisdb = os.path.join(os.path.expanduser("~"), "grassData2/FUTURES_triangle")



# path to the GRASS GIS launch script
# we assume that the GRASS GIS start script is available and on PATH
# query GRASS itself for its GISBASE
# (with fixes for specific platforms)
# needs to be edited by the user

'''
Example: finding grass executable file 
$ locate -b -r ^grass76$
/usr/bin/grass76 <--
/usr/lib/grass76
/usr/share/grass76
it can be set using the absolute path if it is not in the path
grass7bin = '/usr/bin/grass76'
or 
grass7bin = 'grass76'
'''

#/usr/bin/grass78

#grass7bin = 'grass76'
grass7bin = 'grass78'

startcmd = [grass7bin, '--config', 'path']

try:
    p = subprocess.Popen(startcmd, shell=False,
                         stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    out, err = p.communicate()
except OSError as error:
    sys.exit("ERROR: Cannot find GRASS GIS start script"
             " {cmd}: {error}".format(cmd=startcmd[0], error=error))
if p.returncode != 0:
    sys.exit("ERROR: Issues running GRASS GIS start script"
             " {cmd}: {error}"
             .format(cmd=' '.join(startcmd), error=err))
    
#gisbase = out.strip(os.linesep)
gisbase = str(out).replace("\\n'", "").replace("b'", "")
#gisbase = "/usr/lib/grass78"


# set GISBASE environment variable
os.environ['GISBASE'] = gisbase
    
# define GRASS-Python environment
grass_pydir = os.path.join(gisbase, "etc", "python")
sys.path.append(grass_pydir)

# import (some) GRASS Python bindings
import grass.script as gs
from grass.script import core as grass
#import grass.script as gs
import grass.script.setup as gsetup

#https://grasswiki.osgeo.org/wiki/GRASS_Python_Scripting_Library

import grass.script.core as gcore
import grass.script.raster as grast
import grass.script.utils as gutils
from grass.exceptions import CalledModuleError


#Instalados en python 2
from grass.pygrass.modules.shortcuts import raster as r
from grass.pygrass.modules import Module

# launch session
rcfile = gsetup.init(gisbase, gisdb, location, mapset)

print("Example Calls")
gs.message('Current GRASS GIS 7 environment:')
print(gs.gisenv())

gs.message('Available raster maps:')
for rast in gs.list_strings(type='raster'):
    print(rast)

gs.message('Available vector maps:')
for vect in gs.list_strings(type='vector'):
    print(vect)
    

Example Calls
{'GISDBASE': '/cgeo_home/rlopez3', 'LOCATION_NAME': 'BrazilREDD', 'MAPSET': 'PERMANENT'}
BD_indigenous@PERMANENT
BR_elevation@PERMANENT
BR_landforms@PERMANENT
BR_slope@PERMANENT
BR_states2@PERMANENT
BrazilMask1@PERMANENT
Loss8520@PERMANENT
MASK@PERMANENT
a_loss1985_dist@PERMANENT
aloss1985_distance@PERMANENT
deforestation_by_state@PERMANENT
dist_from_loss1986@PERMANENT
loss1985@PERMANENT
loss1985_dist@PERMANENT
loss1985dist_to_non_forest@PERMANENT
loss1986@PERMANENT
loss1986_dist@PERMANENT
loss1986dist_to_non_forest@PERMANENT
loss1987@PERMANENT
loss1987_dist@PERMANENT
loss1987dist_to_non_forest@PERMANENT
loss1988@PERMANENT
loss1988_dist@PERMANENT
loss1988dist_to_non_forest@PERMANENT
loss1989@PERMANENT
loss1989_dist@PERMANENT
loss1989dist_to_non_forest@PERMANENT
loss1990@PERMANENT
loss1990_dist@PERMANENT
loss1990dist_to_non_forest@PERMANENT
loss1991@PERMANENT
loss1991_dist@PERMANENT
loss1991dist_to_non_forest@PERMANENT
loss1992@PERMANENT
loss1992_dist@PERMANENT
loss1992dis

Current GRASS GIS 7 environment:
Available raster maps:
Available vector maps:


In [4]:
gs.run_command('g.region', raster='MASK@PERMANENT')
print(gs.read_command('g.region', flags='p'))

projection: 99 (Normal Aspect Cylindrical Equal Area)
zone:       0
datum:      wgs84
ellipsoid:  wgs84
north:      620000
south:      -3668000
west:       -7988000
east:       -3700000
nsres:      1000
ewres:      1000
rows:       4288
cols:       4288
cells:      18386944



In [ ]:
#BR_slope@PERMANENT * mask_bahia@PERMANENT 

In [8]:
gs.run_command('g.region', raster='mask_bahia@PERMANENT')
print(gs.read_command('g.region', flags='p'))

projection: 99 (Normal Aspect Cylindrical Equal Area)
zone:       0
datum:      wgs84
ellipsoid:  wgs84
north:      620000
south:      -3668000
west:       -7988000
east:       -3700000
nsres:      1000
ewres:      1000
rows:       4288
cols:       4288
cells:      18386944



In [ ]:
#cut slopes map with bahia mask using mapcalc

gs.run_command('r.out.xyz', input=s, output="bahia_slopes.csv", separator = ",")

In [54]:
print("Substituing 0 for nulls")
for m in forest_loss:
    print("MAP {n}".format(n=m))
    gs.run_command('r.null',map=m, setnull='0')

Substituing 0 for nulls
MAP loss1985@PERMANENT


   0   2   4   6   8  10  12  14  16  18  20  22  24  26  28  30  32  34  36  38  40  42  44  46  48  50  52  54  56  58  60  62  64  66  68  70  72  74  76  78  80  82  84  86  88  90  92  94  96  98 100


MAP loss1986@PERMANENT


   0   2   4   6   8  10  12  14  16  18  20  22  24  26  28  30  32  34  36  38  40  42  44  46  48  50  52  54  56  58  60  62  64  66  68  70  72  74  76  78  80  82  84  86  88  90  92  94  96  98 100


MAP loss1987@PERMANENT


...

   0   2   4   6   8  10  12  14  16  18  20  22  24  26  28  30  32  34  36  38  40  42  44  46  48  50  52  54  56  58  60  62  64  66  68  70  72  74  76  78  80  82  84  86  88  90  92  94  96  98 100


MAP loss2019@PERMANENT


   0   2   4   6   8  10  12  14  16  18  20  22  24  26  28  30  32  34  36  38  40  42  44  46  48  50  52  54  56  58  60  62  64  66  68  70  72  74  76  78  80  82  84  86  88  90  92  94  96  98 100


MAP loss2020@PERMANENT


   0   2   4   6   8  10  12  14  16  18  20  22  24  26  28  30  32  34  36  38  40  42  44  46  48  50  52  54  56  58  60  62  64  66  68  70  72  74  76  78  80  82  84  86  88  90  92  94  96  98 100


In [56]:
inp = "BR_elevation"
output =  "BR_slope"
gs.run_command('r.slope.aspect',elevation=inp, slope = output,overwrite='True')


   0   3   6   9  12  15  18  21  24  27  30  33  36  39  42  45  48  51  54  57  60  63  66  69  72  75  78  81  84  87  90  93  96  99 100
Slope raster map <BR_slope> complete


0

In [55]:
for m in forest_loss:
    n = m[:8]+"_"+"dist"
    gs.run_command('r.grow.distance', input = m, distance = n, overwrite = 'True')
    

Reading raster map <loss1985@PERMANENT>...
   0   3   6   9  12  15  18  21  24  27  30  33  36  39  42  45  48  51  54  57  60  63  66  69  72  75  78  81  84  87  90  93  96  99 100
Writing output raster maps...
   0   3   6   9  12  15  18  21  24  27  30  33  36  39  42  45  48  51  54  57  60  63  66  69  72  75  78  81  84  87  90  93  96  99 100
Reading raster map <loss1986@PERMANENT>...
   0   3   6   9  12  15  18  21  24  27  30  33  36  39  42  45  48  51  54  57  60  63  66  69  72  75  78  81  84  87  90  93  96  99 100
Writing output raster maps...
   0   3   6   9  12  15  18  21  24  27  30  33  36  39  42  45  48  51  54  57  60  63  66  69  72  75  78  81  84  87  90  93  96  99 100
Reading raster map <loss1987@PERMANENT>...
   0   3   6   9  12  15  18  21  24  27  30  33  36  39  42  45  48  51  54  57  60  63  66  69  72  75  78  81  84  87  90  93  96  99 100
Writing output raster maps...
   0   3   6   9  12  15  18  21  24  27  30  33  36  39  42  45  48  51  54

Exporting distance to non forest

projection: 99 (Normal Aspect Cylindrical Equal Area)
zone:       0
datum:      wgs84
ellipsoid:  wgs84
north:      620000
south:      -3668000
west:       -7988000
east:       -3700000
nsres:      1000
ewres:      1000
rows:       4288
cols:       4288
cells:      18386944



In [80]:
s = forest_loss[0][:8]+"_"+"dist"+"@PERMANENT"
for m in forest_loss[1:]:
    n = m[:8]+"_"+"dist"+"@PERMANENT"
    s = s+','+n

s = s+','+ "BR_slope@PERMANENT,BR_elevation@PERMANENT,BR_landforms@PERMANENT"

s

'loss1985_dist@PERMANENT,loss1986_dist@PERMANENT,loss1987_dist@PERMANENT,loss1988_dist@PERMANENT,loss1989_dist@PERMANENT,loss1990_dist@PERMANENT,loss1991_dist@PERMANENT,loss1992_dist@PERMANENT,loss1993_dist@PERMANENT,loss1994_dist@PERMANENT,loss1995_dist@PERMANENT,loss1996_dist@PERMANENT,loss1997_dist@PERMANENT,loss1998_dist@PERMANENT,loss1999_dist@PERMANENT,loss2000_dist@PERMANENT,loss2001_dist@PERMANENT,loss2002_dist@PERMANENT,loss2003_dist@PERMANENT,loss2004_dist@PERMANENT,loss2005_dist@PERMANENT,loss2006_dist@PERMANENT,loss2007_dist@PERMANENT,loss2008_dist@PERMANENT,loss2009_dist@PERMANENT,loss2010_dist@PERMANENT,loss2011_dist@PERMANENT,loss2012_dist@PERMANENT,loss2013_dist@PERMANENT,loss2014_dist@PERMANENT,loss2015_dist@PERMANENT,loss2016_dist@PERMANENT,loss2017_dist@PERMANENT,loss2018_dist@PERMANENT,loss2019_dist@PERMANENT,loss2020_dist@PERMANENT,BR_slope@PERMANENT,BR_elevation@PERMANENT,BR_landforms@PERMANENT'

In [82]:
gs.run_command('r.out.xyz', input=s, output="bahia_loss1985_2020.csv", separator = ",")

   0   3   6   9  12  15  18  21  24  27  30  33  36  39  42  45  48  51  54  57  60  63  66  69  72  75  78  81  84  87  90  93  96  99 100


0

In [83]:
2020-1985

35

In [52]:
#g.region raster=elev_lid792_1m -p
#r.out.xyz input=elev_lid792_1m output=elev_lid792_1m.csv separator=","

'loss2020-dist'

In [44]:
"dit_to_non_forest_"+"a"

'dit_to_non_foresta'

In [ ]:
gs.run_command('r.grow.distance', input = protected, distance='dist_to_protected', overwrite = 'True')

In [ ]:



    
'''
Reference:
----------
https://grass.osgeo.org/grass76/manuals/libpython/script.html#module-script.setup
'''
r.calib = Module('r.futures.calib')
 
 #g.region raster=landuse_2011 -
#g.region(res="1", flags="p", **compregion)
 
 #https://linuxize.com/post/how-to-install-pip-on-ubuntu-18.04/
#gscript.region(res="1", flags="p", "landuse_2011")
gs.run_command('g.region', raster='landuse_2011')
print(gs.read_command('g.region', flags='p'))

'''
v.to.rast input=protected_areas output=protected_areas use=val
r.null map=protected_areas null=0
'''

gs.run_command('v.to.rast', input='protected_areas', output='protected_areas', use='val',overwrite ='True')
gs.run_command('r.null',map='protected_areas',null='0')

#r.mapcalc "urban_1992 = if(landuse_1992 >= 21 && landuse_1992 <= 24, 1, if(landuse_1992 == 11 || landuse_1992 >= 90 || protected_areas, null(), 0))"
#r.mapcalc "urban_2001 = if(landuse_2001 >= 21 && landuse_2001 <= 24, 1, if(landuse_2001 == 11 || landuse_2001 >= 90 || protected_areas, null(), 0))"
#r.mapcalc "urban_2006 = if(landuse_2006 >= 21 && landuse_2006 <= 24, 1, if(landuse_2006 == 11 || landuse_2006 >= 90 || protected_areas, null(), 0))"
#r.mapcalc "urban_2011 = if(landuse_2011 >= 21 && landuse_2011 <= 24, 1, if(landuse_2011 == 11 || landuse_2011 >= 90 || protected_areas, null(), 0))"


rast_out_1992 = "urban_1992"
rast_out_2001 = "urban_2001"
rast_out_2006 = "urban_2006"
rast_out_2011 = "urban_2011"
protected = "protected_areas"

rast_in_landuse_1992 = "landuse_1992"
rast_in_landuse_2001 = "landuse_2001"
rast_in_landuse_2006 = "landuse_2006"
rast_in_landuse_2011 = "landuse_2011"

counties = "counties"


gs.mapcalc("${out} = if(${rast1} >= 21 && ${rast1} <= 24, 1, if(${rast1} == 11 || ${rast1} >= 90 || ${rast2}, null(), 0))", out = rast_out_1992, rast1 = rast_in_landuse_1992,rast2 = protected,overwrite = "True")
gs.mapcalc("${out} = if(${rast1} >= 21 && ${rast1} <= 24, 1, if(${rast1} == 11 || ${rast1} >= 90 || ${rast2}, null(), 0))", out = rast_out_2001, rast1 = rast_in_landuse_2001,rast2 = protected,overwrite = "True")
gs.mapcalc("${out} = if(${rast1} >= 21 && ${rast1} <= 24, 1, if(${rast1} == 11 || ${rast1} >= 90 || ${rast2}, null(), 0))", out = rast_out_2006, rast1 = rast_in_landuse_2006,rast2 = protected,overwrite = "True")
gs.mapcalc("${out} = if(${rast1} >= 21 && ${rast1} <= 24, 1, if(${rast1} == 11 || ${rast1} >= 90 || ${rast2}, null(), 0))", out = rast_out_2011, rast1 = rast_in_landuse_2011,rast2 = protected,overwrite = "True")

#v.to.rast input=counties type=area use=attr attribute_column=FIPS output=counties

gs.run_command('v.to.rast', input=counties,type = "area", use = 'attr',attribute_column="FIPS", output=counties,overwrite ='True')

input = 'elevation_30m'
output = 'slope'

#r.slope.aspect elevation=elevation_30m slope=slope

gs.run_command('r.slope.aspect',elevation=input, slope = output,overwrite='True')

#Distance from protected areas

'''
r.null map=protected_areas setnull=0
r.grow.distance input=protected_areas distance=dist_to_protected
r.colors map=dist_to_protected color=gyr
'''

gs.run_command('r.null',map=protected, setnull=0,overwrite = 'True')

gs.run_command('r.grow.distance', input = protected, distance='dist_to_protected', overwrite = 'True')

gs.run_command('r.grow.distance', input = "water_nc", distance='dist_to_water', overwrite = 'True')

gs.run_command('r.colors', map = 'dist_to_protected', color='gyr', overwrite = 'True')



#Dist to interchanges

'''
v.to.rast input=roads type=line where="MTFCC = 'S1630'" output=interchanges use=val
r.grow.distance -m input=interchanges distance=dist_interchanges
'''

gs.run_command('v.to.rast',input='roads', type='line', where="MTFCC = 'S1630'", output = 'interchanges', use = 'val',overwrite = "True")


gs.run_command('r.grow.distance', input='interchanges',distance='dist_interchanges',flags = 'm',overwrite = "True")

#Forests

'''
r.mapcalc "forest = if(landuse_2011 >= 41 && landuse_2011 <= 43, 1, 0)"
r.mapcalc "forest_1992 = if(landuse_1992 >= 41 && landuse_1992 <= 43, 1, 0)"
r.neighbors -c input=forest output=forest_smooth size=15 method=average
r.neighbors -c input=forest_1992 output=forest_1992_smooth size=15 method=average
r.colors map=forest_smooth,forest_1992_smooth color=ndvi
'''

gs.mapcalc('forest = if(landuse_2011 >= 41 && landuse_2011 <= 43, 1, 0)', overwrite='True')

gs.mapcalc('forest_1992 = if(landuse_1992 >= 41 && landuse_1992 <= 43, 1, 0)' , overwrite='True')



gs.run_command('r.neighbors', input='forest',output='forest_smooth', size='15', method='average', flags = 'c', overwrite = 'True')

gs.run_command('r.neighbors', input='forest_1992',output='forest_1992_smooth', size='15', method='average', flags = 'c', overwrite = 'True')

gs.run_command('r.colors', map = 'forest_smooth', color = 'ndvi')

gs.run_command('r.colors', map = 'forest_1992_smooth', color = 'ndvi')

#Road density

'''
v.to.rast input=roads output=roads use=val type=line
r.null map=roads null=0
r.neighbors -c input=roads output=road_dens size=25 method=average
'''

gs.run_command('v.to.rast',input = 'roads',output='roads', use = 'val', type= 'line', overwrite = 'True')

gs.run_command('r.null', map = 'roads', null='0', overwrite = 'True')

gs.run_command('r.neighbors', input = 'roads', output='road_dens', size = '25', method = 'average', flags = 'c', overwrite = 'True')

#https://grasswiki.osgeo.org/wiki/Workshop_on_urban_growth_modeling_with_FUTURES

# https://ncsu-geoforall-lab.github.io/uav-lidar-analytics-course/assignments/python_intro.html
#Lakesand rivers

# time to cities

#dev pressure
gs.run_command('r.futures.devpressure', input = 'urban_1992', output = 'devpressure_0_5_92',method = 'gravity', size = '30', gamma = '0.5', scaling_factor='0.1', flags = 'n', overwrite='True')


gs.run_command('v.db.addcolumn',map='roadsc',columns="speed double precision", overwrite='True')

gs.run_command('v.db.update',map='roadsc',column="speed", value=50,where="MTFCC = 'S1100'" , overwrite='True')
gs.run_command('v.db.update',map='roadsc',column="speed", value=100,where="MTFCC = 'S1200'" , overwrite='True')
gs.run_command('v.db.update',map='roadsc',column="speed", value=100,where="MTFCC = 'S1400'" , overwrite='True')
'''
 Attribute MTFCC 	speed [km/h]
S1400 	50
S1100 	100
S1200 	100 
'''

#v.db.update map=myroads column=speed value=50 where="MTFCC = 'S1400'"

#v.db.update map=myroads column=speed value=50 where="MTFCC = 'S1400'"


#v.db.update map=roadsc column=speed value=100 where="MTFCC IN ('S1100', 'S1200')"

#
#devpressure_0_5_92
#road_dens_perc
#forest_1992_smooth_perc	
#dist_to_water_km
#dist_to_protected_km

gs.run_command("v.to.rast",input='roadsc', type='line', where = "MTFCC IN ('S1100', 'S1200', 'S1400')", output='roads_speed', use='attr', attribute_column='speed',overwrite=True)
#                v.to.rast input=roadsc type=line where="MTFCC IN ('S1100', 'S1200', 'S1400')" output=roads_speed use=attr attribute_column=speed





gs.mapcalc("dist_to_water_km = dist_to_water / 1000",overwrite='True')
gs.mapcalc("dist_to_protected_km = dist_to_protected / 1000",overwrite='True')
gs.mapcalc("dist_to_interchanges_km = dist_interchanges / 1000",overwrite='True')
gs.mapcalc("road_dens_perc = road_dens * 100",overwrite='True')
gs.mapcalc("forest_smooth_perc = forest_smooth * 100",overwrite='True')

gs.run_command("r.null", map = "roads_speed", null = "5", overwrite = "True")
gs.mapcalc("roads_travel_time = 1.8 / roads_speed", overwrite = "True")

gs.mapcalc("urban_2011_clip = if(counties, urban_2011)", overwrite = 'True')

gs.run_command("r.report", map = "urban_2001", units = "h,c,p", output = "report.txt", overwrite =  'True')


gs.run_command("r.futures.devpressure", flags = "n", input="urban_2011", output='devpressure_0_5', method = 'gravity', size = "30", gamma = '0.5', scaling_factor = "0.1", overwrite = 'True')

'''
r.futures.devpressure -n input=urban_2011 output=devpressure_0_5 method=gravity size=30 gamma=0.5 scaling_factor=0.1
r.sample.category input=urban_2011_clip output=sampling sampled=counties,devpressure_0_5,devpressure_1,slope,road_dens_perc,forest_smooth_perc,dist_to_water_km,dist_to_protected_km,dist_interchanges_km,travel_time_cities npoints=10000,2000
'''

gs.mapcalc("urban_2011_clip = if(counties, urban_2011)", overwrite = 'True')


#r.sample.category input=urban_2011_clip output=sampling sampled=counties,devpressure_0_5,slope,road_dens_perc,forest_smooth_perc,dist_to_water_km,dist_to_protected_km,dist_to_interchanges_km,travel_time_cities npoints=10000,2000 --o
#r.futures.potential  input=sampling output=potential.csv columns=devpressure_0_5,slope,forest_smooth_perc,dist_to_interchanges_km,travel_time_cities developed_column=urban_2011_clip subregions_column=counties --o
gs.run_command("r.sample.category",output="sampling", input="urban_2011_clip",sampled = "counties,devpressure_0_5,slope,road_dens_perc,forest_smooth_perc,dist_to_water_km,dist_to_protected_km,dist_to_interchanges_km,travel_time_cities", npoints="10000,2000", overwrite = 'True')




gs.run_command("r.cost", input="roads_travel_time", output="travel_time_cities", start_points = "cities", overwrite = "True")


#gs.run_command("v.to.rast", input="roads_travel_time", output="travel_time_cities", start_points="cities_20000" )

gs.run_command("r.futures.potential",  input="sampling", output="potential.csv", columns="counties,devpressure_0_5,slope,road_dens_perc,forest_smooth_perc,dist_to_water_km,dist_to_protected_km,dist_to_interchanges_km,travel_time_cities", developed_column="urban_2011_clip", subregions_column="counties", overwrite = "True")


gs.run_command("v.to.rast", input="roads", type="line", output="roads_mask", use="val", overwrite= "True")

#r.mask roads_mask -i
gs.run_command("r.mask", raster="roads_mask", flags = "i", overwrite = 'True')

#r.futures.demand development=urban_1992,urban_2001,urban_2006 subregions=counties observed_population=/home/rdglpz/evaluation_pga_parameters/population_trend.csv projected_population=/home/rdglpz/evaluation_pga_parameters/population_projection.csv simulation_times=2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035 plot=plot_demand.pdf demand=demand.csv separator=comma


#r.futures.potential input=sampling output=potential.csv columns=devpressure_0_5,slope,forest_smooth_perc,dist_to_interchanges_km,travel_time_cities developed_column=urban_2011_clip subregions_column=counties --o

#r.futures.demand development=urban_2001,urban_2006,urban_2011 subregions=counties observed_population=/home/rdglpz/evaluation_pga_parameters/population_trend.csv projected_population=/home/rdglpz/evaluation_pga_parameters/population_projection.csv simulation_times=2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035 plot=plot_demand.pdf demand=demand.csv separator=comma
'''
r.futures.demand development=urban_2001,urban_2006,urban_2011 subregions=counties observed_population=/home/rdglpz/evaluation_pga_parameters/population_trend.csv projected_population=/home/rdglpz/evaluation_pga_parameters/population_projection.csv simulation_times=2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035 plot=plot_demand.pdf demand=demand.csv separator=comma
'''

'''
v.to.rast input=roads output=roads use=val type=line
v.to.rast
r.cost input=roads_travel_time output=travel_time_cities start_points=cities_20000
r.colors map=travel_time_cities color=byr
'''


'''
r.null map=roads_speed null=5
r.mapcalc ""
'''

'''
r.sample.category input=urban_2011_clip output=sampling sampled=counties,devpressure_0_5,slope,road_dens_perc,forest_smooth_perc,dist_to_water_km,dist_to_protected_km,dist_to_interchanges_km,travel_time_cities npoints=10000,2000 --o
'''

#v.voronoi input=counties_random_n10b output=vorono_37021

#v.to.rast voronoi_37021 out=voronoi_37021_rast use=cat

#r.colors voronoi_37021_rast color=random



#r.mapcalc "vor_landuse_conty = if(voronoi_37021_rast == 1 && counties == 37021 , devpressure_0_5_92, 0)"


#gs.run_command("r.null",map="protected_areas",setnull=0)
#r.null map=protected_areas setnull=0
#r.grow.distance input=protected_areas distance=dist_to_protected
#r.colors map=dist_to_protected color=gyr -e
               
               # Referencias
#https://grasswiki.osgeo.org/wiki/Workshop_on_urban_growth_modeling_with_FUTURES
#https://grasswiki.osgeo.org/wiki/FUTURES_tutorial